### Task 3

В этом задании мы создадим витрину данных, объединяя несколько таблиц, для дальнейшего анализа этих данных \
Нас интересуют следующие данные: даты первых коммитов заданий, дата первого посещения ленты новостей и др.

1. Используя библиотеку *sqlite3*, создадим соединение с базой данных

In [1]:
import sqlite3

db_con = sqlite3.connect('..\..\data\checking-logs.sqlite')

2. Используя только *один* запрос, объединим таблицы `pageviews` и `checker` в таблицу **datamart**, удовлетворящую следующим условиям:
    - Таблица должна иметь столбцы - `uid`, `labname`, `first_commit_ts`, `first_view_ts`, где
        - `first_commit_ts` - это новое имя столбца `timestamp` из таблицы `checker`, который содержит дату первого коммита каждого задания каждого учащегося
        - `first_view_ts` -  это новое имя столбца `datetime` из таблицы `pageviews`, который содержит дату первого посещения учащимся ленты новостей
    - Так же рассматриваем только готовые задания `status = 'ready'`
    - Так же рассматриваем только первые коммиты `numTrials = 1`
    - Так же нас интересуют только задания: `laba04`, `laba04s`, `laba05`, `laba06`, `laba06s`, `project1`
    - Так же нас интересуют только учащиеся `uid = user_*`
    - `first_commit_ts` и `first_view_ts` нужно распарсить как *datetime64[ns]*

In [2]:
import pandas as pd

datamart = pd.io.sql.read_sql('SELECT ch.uid, ch.labname, ch.timestamp AS first_commit_ts, MIN(pag.datetime) AS first_view_ts\
                              FROM checker AS ch\
                              LEFT JOIN pageviews AS pag\
                              ON ch.uid = pag.uid\
                              WHERE ch.status = "ready" AND\
                              ch.numTrials = 1 AND\
                              ch.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s", "project1") AND\
                              ch.uid LIKE "user_%"\
                              GROUP BY ch.timestamp, ch.uid', db_con,  parse_dates=['first_commit_ts','first_view_ts']) # распарсены как datetime64[ns]

datamart.to_sql('datamart', db_con, if_exists='replace')  # сохранили df в БД 

datamart.info()
#datamart

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


3. Создадим два датафрейма **test** и **control**, с помощью методов библиотеки **Pandas**
    - в `test` включаем учащихся, у которых есть значения в `first_view_ts`, т.е. кто посещал ленту новостей
    - в `control` включаем учащихся, у которых нет значения в `first_view_ts`, т.е. кто не посещал ленту новостей
    - отсутствующие значения `first_view_ts` в `control` заменим на среднее значение `first_view_ts` из `test`, в дальнейшем будем использовать его для анализа
    - сохраним **test** и **control** в базе данных

In [3]:
test = datamart[datamart['first_view_ts'].notnull()] 
control = datamart[datamart['first_view_ts'].isnull()]

control.loc[:, 'first_view_ts'] = test['first_view_ts'].mean()

test.to_sql('test', db_con, if_exists='replace') # сохранили оба df в БД 
control.to_sql('control', db_con, if_exists='replace')

test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [4]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


4. Закроим соединение с базой данных

In [ ]:
db_con.close()